In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

In [ ]:
import numpy as np

### Задание 1
Прочитайте базу "Потребление Энергии" и сделайте модель для сезонных изменений по data_exp_1 с горизонт прогноза h=1 и h=10 по образцу из примера для обработки ряда "цена Нефти" . Учитывать только глубину истории при p=3

In [ ]:
data = pd.read_csv('my_PJME_MW.csv', index_col=[0], parse_dates=[0])
data.head()

In [ ]:
data.info()

In [ ]:
data = pd.DataFrame(data = data[['PJME_MW']])
data.head()

In [ ]:
plt.figure(figsize =(20,4))
plt.plot(data.index, data['PJME_MW'], 'b' )
plt.title('PJM East потребление энергии' )
plt.ylabel ( 'МВт' )
plt.show()

In [ ]:
def exponential_smoothing(series, alpha):
    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result

In [ ]:
data_day = data.resample('D').mean()

In [ ]:
data_exp_smooth = exponential_smoothing(data_day['PJME_MW'], 0.05)

In [ ]:
data_exp_smooth = pd.DataFrame(data=data_exp_smooth, index=data_day.index)

In [ ]:
plt.figure(figsize =(20,4))
plt.plot(data_exp_smooth.index, data_exp_smooth[0], 'b' )
plt.title('PJM East с экспоненциальным сглаживанием и дневным ресэмплом' )
plt.ylabel ( 'МВт' )
plt.show()

In [ ]:
df = pd.DataFrame(data=data_exp_smooth)

In [ ]:
df.rename(columns={0: 'PJME_MW'}, inplace=True)
df.head()

In [ ]:
df['t'] = 0
df.iloc[1:, 1] = df.iloc[:-1, 0].values
df['t1'] = 0
df.iloc[2:, 2] = df.iloc[:-2, 0].values
df['t2'] = 0
df.iloc[3:, 3] = df.iloc[:-3, 0].values
df.head()

In [ ]:
def get_split_data(data, split_date):
    return data[:split_date], data[split_date:]

In [ ]:
splitter = '2017-01-01'
train, test = get_split_data(df, splitter)

In [ ]:
plt.figure( figsize=( 15, 7 ))
plt.xlabel('Время')
plt.ylabel('Цена нефти Brent')
plt.plot(train.index,train[df.columns[0]], label='train data' )
plt.plot(test.index, test[df.columns[0]], label='test data')
plt.title( 'Тестовые и тренировочные данные' )
plt.legend()
plt.show()

In [ ]:
def mean_absolute_percentage_error(y, y_pred):
    y, y_pred = np.array(y), np.array(y_pred)
    return np.mean(np.abs(y - y_pred) / y) * 100

Обучим модель для предсказания с горизонтом прогноза 1

In [ ]:
h = 1
X_train = train.iloc[:-h,:]
y_train = train['PJME_MW'].values[h:]
X_test = test.iloc[:-h,:]
y_test = test['PJME_MW'].values[h:]

In [ ]:
model_gb = GradientBoostingRegressor(max_depth=15, random_state=0, n_estimators=100)
model_gb.fit( X_train, y_train) 
X_test_pred_gb = model_gb.predict(X_test)

er_g_1 = mean_absolute_percentage_error(y_test, X_test_pred_gb)

print(f'ошибка градиентного бустинга : {er_g_1:.2}%')

In [ ]:
plt.figure( figsize=( 20, 4 ))

plt.xlabel('Время')
plt.ylabel('Потребление энергии')
plt.plot(test.index[h:],X_test_pred_gb, label='predict GB data' )
plt.plot(test.index[h:],y_test, label='test data')

plt.legend()
plt.show()

Обучим модель для предсказания с горизонтом прогноза 10

In [ ]:
h = 10
X_train = train.iloc[:-h,:]
y_train = train['PJME_MW'].values[h:]
X_test = test.iloc[:-h,:]
y_test = test['PJME_MW'].values[h:]

In [ ]:
model_gb = GradientBoostingRegressor(max_depth=15, random_state=0, n_estimators=100)
model_gb.fit( X_train, y_train) 
X_test_pred_gb = model_gb.predict(X_test)

er_g_10 = mean_absolute_percentage_error(y_test, X_test_pred_gb)

print(f'ошибка градиентного бустинга : {er_g_10:.2}%')

In [ ]:
plt.figure( figsize=( 20, 4 ))

plt.xlabel('Время')
plt.ylabel('Потребление энергии')
plt.plot(test.index[h:],X_test_pred_gb, label='predict GB data' )
plt.plot(test.index[h:],y_test, label='test data')

plt.legend()
plt.show()

### Задание 2
Оцените ошибку модели

In [ ]:
print(f'H=1, ошибка градиентного бустинга : {er_g_1:.2}%')
print(f'H=10, ошибка градиентного бустинга : {er_g_10:.2}%')

### Задание 3
Сделайте вывод о том, как изменилась ошибка модели по значениям h.

С увеличением горизонта прогноза ошибка прогноза предсказуемо растет.

### Задание 4
Попробуйте предположить характер ряда (стационарный или нет, с трендом, с сезоном, с циклом) для рядов из списка и аргументируйте свой ответ (одна строка):  
а) Изменение температуры за окном :).  
б) Количество денег, которые тратят покупатели в интернет магазине.  
в) Вес растущего котенка.  
г) Прирост объема деревьев в лесу от времени  

а) не стационарный, сезонный, если верить в глобальное потепление то есть тренд на увеличение.  
б) не стационарный, сезонный, цикличный (цикл привязан к циклам в экономике), а также есть тренд на увиличение для стран с заметной инфляцией.  
в) не стационарный, есть тренд на увеличение  
г) предполагаю что рост количества деревьев происходит в некой геометрической прогрессии, тогда их прирост будет не стационарным с трендом на увеличение, в странах с холодным временем года сезоннный.  

### Задание 5
Какие из рядов на картинке стационарны на первый взгляд? (презентация, слайд номер 21)

b d